In [78]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import os
import sklearn
import cv2

In [79]:
# 데이터가 많으므로 테스트에서는 1000건만 처리하도록 한다.
NUM_ROWS = 1000

DATASET_PATH = 'C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷'
print(os.listdir(DATASET_PATH))

['상의']


In [84]:
categories = []
filenames = []
images = []

pd.set_option('display.max_colwidth', None) # None을 사용하면 전체 내용이 표시됩니다.

df = pd.DataFrame(columns=['filename', 'image'])
for folder in os.listdir(DATASET_PATH):
    folder_path = os.path.join(DATASET_PATH, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        filenames.append(file)
        images.append(os.path.join(DATASET_PATH, folder, file).replace('\\','/'))


df['filename'] = filenames
df['image'] = images

df = df[:NUM_ROWS]

df.head(10)

,filename,image
0,0.jpg,C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/0.jpg
1,1.jpg,C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/1.jpg
2,10.jpg,C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/10.jpg
3,100.jpg,C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/100.jpg
4,101.jpg,C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/101.jpg
5,102.jpg,C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/102.jpg
6,103.jpg,C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/103.jpg
7,104.jpg,C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/104.jpg
8,105.jpg,C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/105.jpg
9,106.jpg,C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/106.jpg


In [81]:
# 딕셔너리 데이터를 인자값을 받아서 이미지를 출력한다.
def plot_figures(figures, nrows = 1, ncols=1,figsize=(5, 3)):
    fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows, figsize=figsize)
    for ind,title in enumerate(figures):
        axeslist.ravel()[ind].imshow(cv2.cvtColor(figures[title], cv2.COLOR_BGR2RGB))
        axeslist.ravel()[ind].set_title(title)
        axeslist.ravel()[ind].set_axis_off()
    plt.tight_layout()

In [82]:
# 이미지 경로를 읽어서 numpy로 변환한다.
# 기본 0.1 비율로 리사이즈를 처리한다.
def load_image(img_path, resized_fac = 0.1):
    img  = cv2.imread(img_path)
    w, h, _ = img.shape
    resized = cv2.resize(img, (int(h*resized_fac), int(w*resized_fac)), interpolation = cv2.INTER_AREA)
    return resized


In [83]:
figures = {'sample'+ str(i) : load_image(row.image) for i, row in df.sample(6).iterrows()}
figures.keys()
plot_figures(figures, 2, 3)

AttributeError: 'NoneType' object has no attribute 'shape'

In [31]:
import tensorflow as tf
import keras
from keras import Model
from tensorflow.keras.applications.resnet50 import ResNet50
# from keras.preprocessing import image
import keras.utils as image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import GlobalMaxPooling2D
tf.__version__

'2.13.0'

In [32]:
# 입력 이미지

img_width, img_height, img_channel = 224, 224, 3

# 훈련된 모델 사용
base_model = ResNet50(weights='imagenet', include_top=False, input_shape = (img_width, img_height, img_channel))
base_model.trainable = False

model = keras.Sequential([
    base_model,
    GlobalMaxPooling2D()
])

model.summary()

94765736/94765736 [==============================] - 8s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Glob  (None, 2048)              0         
 alMaxPooling2D)                                                 
                                                                 
Total params: 23587712 (89.98 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [33]:
# 전체 데이터 임베딩
def get_embedding(model, img_path):
    # Pillow 이미지 로드
    img = image.load_img(img_path, target_size=(img_width, img_height))
    # numpy 데이터로 변환 (224, 224, 3)
    x = image.img_to_array(img)
    # 차원을 추가해준다. (1, 224, 224, 3)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    # 1차원의 배열로 재배열해준다. [[5.232, 2.12, ...]] -> [5.232, 2.12, ...]
    return model.predict(x).reshape(-1)

In [34]:
df.iloc[0].image

'C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷\\상의\\523.jpg'

In [35]:
# 첫 행 이미지 임베딩을 출력
emb = get_embedding(model, df.iloc[0].image)
emb.shape

1/1 [==============================] - 1s 851ms/step


(2048,)

In [36]:
# 첫 행 이미지 시각화
img_array = load_image(df.iloc[0].image)
plt.figure(figsize = (2,2))
plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
print(img_array.shape)
print(emb)

AttributeError: 'NoneType' object has no attribute 'shape'

In [22]:
%%time
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

# 전체 데이터 임베딩
df_sample = df
map_embeddings = df_sample['image'].progress_apply(lambda img: get_embedding(model, img))
df_embs = map_embeddings.apply(pd.Series)

print(df_embs.shape)
df_embs.head()

0it [00:00, ?it/s]
1it [00:00, 1002.46it/s]


NameError: name 'model' is not defined

In [23]:
# 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

# cosine 거리 계산
pairwise_distances(df_embs, metric='cosine')

# 정규화
cosine_sim = 1 - pairwise_distances(df_embs, metric='cosine')
cosine_sim

In [ ]:
# 유사한 이미지

In [ ]:
indices = pd.Series(range(len(df)), index=df.index)

def get_recommender(idx, df, top_n = 5):
    sim_idx = indices[idx]
    sim_scores = list(enumerate(cosine_sim[sim_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    idx_rec    = [i[0] for i in sim_scores]
    idx_sim    = [i[1] for i in sim_scores]

    return indices.iloc[idx_rec].index, idx_sim

In [ ]:
# 유사한 데이터 출력해보기
idx_ref = 1
idx_rec, idx_sim = get_recommender(idx_ref, df, top_n = 6)

plt.figure(figsize = (2,2))
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

figures = {'img-'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
plot_figures(figures, 2, 3)